In [ ]:
# 下記セルを実行すると、authorization codeの入力を求められます。
# 出力されたリンク先をクリックし、Googleアカウントにログインし、
# authorization codeをコピーし、貼り付けをおこなってください。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
project = 'sample_data'
chapter = 9
os.chdir(f'/content/drive/MyDrive/{project}/chapter-{chapter}/')

# ９章 ChatGPTを用いたデータ加工・可視化を行う10本ノック

## 放課後ノック１０２：数値データの差分を計算してみよう




In [ ]:
import pandas as pd

# CSVファイルのパスを指定
csv_file_path = 'data/receive_time.csv'

# CSVファイルをデータフレームとして読み込む
df = pd.read_csv(csv_file_path)

# データフレームの内容を表示
print(df)


In [ ]:
import pandas as pd

def read_timeseries_csv(file_path, datetime_column):
    """
    CSVファイルをデータフレームとして読み込み、指定された列をdatetime型に変換する関数。

    Parameters:
    - file_path (str): CSVファイルのパス
    - datetime_column (str): datetime型に変換したい列のカラム名

    Returns:
    - pd.DataFrame: 読み込まれたデータフレーム
    """
    # CSVファイルをデータフレームとして読み込む
    df = pd.read_csv(file_path)

    # 指定された列をdatetime型に変換
    df[datetime_column] = pd.to_datetime(df[datetime_column])

    return df

# 対象のファイルパスとdatetime型として読み込みたい列のカラム名を指定
file_path = 'data/receive_time.csv'
datetime_column_name = 'receive_time_sec'

# 関数を使用してデータフレームを作成
df_result = read_timeseries_csv(file_path, datetime_column_name)

# データフレームの内容を表示
display(df_result)

In [ ]:
import pandas as pd

def perform_data_analytics(df):
    """
    指定されたデータフレームに対して、列方向に１行ずらし、差分を計算して新しいデータフレームを作成する関数。

    Parameters:
    - df (pd.DataFrame): 元のデータフレーム

    Returns:
    - pd.DataFrame: 新しいデータフレーム
    """
    # 列方向に１行ずらす
    data_before_1sec = df.shift(1)

    # カラム名を変更
    data_before_1sec.columns = [col + '_b1sec' for col in data_before_1sec.columns]

    # 元のデータフレームと１行ずらしたデータフレームを結合
    data_analytics = pd.concat([df, data_before_1sec], axis=1)

    # 差分を計算して新しいカラムを追加
    data_analytics['in1_calc'] = data_analytics['in1'] - data_analytics['in1_b1sec']
    data_analytics['out1_calc'] = data_analytics['out1'] - data_analytics['out1_b1sec']

    return data_analytics

# 例として作成したデータフレームを使用
# df_result = ...

# 関数を使用してデータ解析を実行
data_analytics_result = perform_data_analytics(df_result)

# 先頭５行を表示
display(data_analytics_result.head())

processed_data = data_analytics_result.copy()

## 放課後ノック１０３：統計量を確認してみよう




In [ ]:
data_analytics_result.describe()

In [ ]:
import pandas as pd

def aggregate_and_melt_data(data_analytics_result):
    """
    指定されたデータフレームを処理し、秒単位の時間データを時単位に集計し、縦持ちのデータフレームに変換する関数。

    Parameters:
    - data_analytics_result (pd.DataFrame): 元のデータフレーム

    Returns:
    - pd.DataFrame: 処理されたデータフレーム
    """
    # datetime型カラムを文字列に変換して新しいカラムに格納
    data_analytics_result['date_hour'] = data_analytics_result['receive_time_sec'].dt.strftime('%Y%m%d%H')

    # カラムを抽出して時単位で集計
    viz_data = data_analytics_result[['date_hour', 'in1_calc', 'out1_calc']].groupby('date_hour').sum().reset_index()

    # 'in1_calc'と'out1_calc'を縦持ちに変換
    viz_data_melted = pd.melt(viz_data, id_vars=['date_hour'], value_vars=['in1_calc', 'out1_calc'],
                              var_name='variable', value_name='value')

    return viz_data_melted

# 例として作成したデータフレームを使用
# data_analytics_result = ...

# 関数を使用してデータ処理を実行
viz_data = aggregate_and_melt_data(data_analytics_result)

# 先頭５行を表示
display(viz_data.head())


## 放課後ノック１０４：時系列の可視化をしてみよう




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# グラフのスタイルを設定（任意）
sns.set(style="whitegrid")

# 折れ線グラフの作成
plt.figure(figsize=(12, 6))  # グラフのサイズを設定

# カラム'variable'の値ごとに折れ線グラフを描画
sns.lineplot(x='date_hour', y='value', hue='variable', data=viz_data)

# グラフにタイトルやラベルを追加（任意）
plt.title('Hourly Aggregated Data Visualization')
plt.xlabel('Hourly Time')
plt.ylabel('Value')

# 凡例を表示
plt.legend(title='Variable')

# グラフを表示
plt.show()

## 放課後ノック１０５：データの分布をヒストグラムで確可視化してみよう




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# グラフのスタイルを設定（任意）
sns.set(style="whitegrid")

# ヒストグラムの作成
plt.figure(figsize=(10, 6))  # グラフのサイズを設定

# カラム'value'のヒストグラムを描画
sns.histplot(data=viz_data, x='value', hue='variable', kde=True)

# グラフにタイトルやラベルを追加（任意）
plt.title('Histogram of Value')
plt.xlabel('Value')
plt.ylabel('Frequency')

# 凡例を表示
plt.legend(title='Variable')

# グラフを表示
plt.show()

## 放課後ノック１０７：文章を単語分割してみよう




In [ ]:
booklist = pd.read_csv('data/hashire_merosu.csv')
booklist

In [ ]:
%%bash

apt install -yq \
  mecab \
  mecab-ipadic-utf8 \
  libmecab-dev
pip install -q mecab-python3
ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
pip list | grep mecab

In [ ]:
import pandas as pd
import MeCab

def mecab_tokenize(text):
    """
    MeCabを使用してテキストを形態素解析し、名詞と動詞だけを抽出する関数。

    Parameters:
    - text (str): 解析対象のテキスト

    Returns:
    - pd.DataFrame: 抽出された単語と品詞のデータフレーム
    """
    # MeCabのインスタンスを生成
    mecab = MeCab.Tagger()

    # テキストを形態素解析して単語と品詞を取得
    parsed_text = mecab.parse(text)

    # 解析結果をデータフレームに格納
    words = []
    pos_tags = []
    for line in parsed_text.splitlines():
        if line == "EOS" or line == "":
            continue
        surface, feature = line.split("\t")
        features = feature.split(",")
        if features[0] == "名詞" or features[0] == "動詞":
            words.append(surface)
            pos_tags.append(features[0])

    # データフレームに変換
    df = pd.DataFrame({'word': words, '種類': pos_tags})

    return df

# 例として作成したデータフレームを使用
# booklist = ...

# 関数を使用して形態素解析とデータフレームの作成を実行
result_df = mecab_tokenize(booklist['body'].iloc[0])

# 先頭５行を表示
print(result_df.head())


## 放課後ノック１０８：単語の使用頻度を可視化してみよう




In [ ]:
count = result_df.groupby('word').size().sort_values(ascending=False)
count.name = 'count'
count = count.reset_index().head(20)
count

In [ ]:
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_top_words(count_df, top_n=20):
    """
    カウントデータフレームから上位N個の単語を横向きの棒グラフで可視化する関数。

    Parameters:
    - count_df (pd.DataFrame): カウントデータフレーム
    - top_n (int): 上位N個の単語を表示する数（デフォルトは20）

    Returns:
    - None
    """
    # カウントが大きい上位N個の単語を取得
    top_words = count_df.nlargest(top_n, 'count')

    # 横向きの棒グラフを描画
    plt.figure(figsize=(10, 8))
    plt.barh(top_words['word'], top_words['count'], color='skyblue')

    # グラフにタイトルやラベルを追加（任意）
    plt.title(f'Top {top_n} Words by Count')
    plt.xlabel('Count')
    plt.ylabel('Word')

    # グラフを表示
    plt.show()

# 例として作成したデータフレームを使用
# count = ...

# 関数を使用して可視化を実行
plot_top_words(count)


## 放課後ノック１０９：文章からエンティティ抽出してみよう




In [ ]:
!pip install ja-ginza

In [ ]:
import spacy
from spacy.lang.ja import Japanese
from spacy.lang.ja.examples import sentences
from IPython.display import display

# spacyのja_ginzaモデルを読み込む
nlp = spacy.load('ja_ginza')

def extract_entities(text):
    """
    spacyを使用して与えられたテキストからエンティティを抽出する関数。

    Parameters:
    - text (str): エンティティを抽出する対象のテキスト

    Returns:
    - pd.DataFrame: 抽出されたエンティティのデータフレーム
    """
    # テキストを処理
    doc = nlp(text)

    # エンティティの抽出
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))

    # データフレームに変換
    entity_df = pd.DataFrame(entities, columns=['Entity', 'Label'])

    return entity_df

# 例として作成したデータフレームを使用
# booklist = ...

# エンティティ抽出を実行
entity_df = extract_entities(booklist['body'].iloc[0])

# 結果のデータフレームを表示
display(entity_df)


## 放課後ノック１１０：エンティティをハイライトで可視化してみよう




In [ ]:
import spacy
from spacy import displacy
from IPython.display import display

# spacyのja_ginzaモデルを読み込む
nlp = spacy.load('ja_ginza')

def highlight_entities(text):
    """
    spacyを使用して与えられたテキストからエンティティを抽出し、ハイライトして表示する関数。

    Parameters:
    - text (str): エンティティを抽出し、ハイライトする対象のテキスト

    Returns:
    - None
    """
    # テキストを処理
    doc = nlp(text)

    # エンティティをハイライトして表示（jupyter=True）
    displacy.render(doc, style='ent', jupyter=True)

# 例として作成したデータフレームを使用
# booklist = ...

# エンティティ抽出とハイライト表示を実行
highlight_entities(booklist['body'].iloc[0])
